<a href="https://colab.research.google.com/github/lkoneha/LLM/blob/main/Bert%2BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
df= pd.read_csv('DS4_WELDA.csv')
df.head()

,API,Description,Category,Categoryl
0,Google App Engine API,[This API is no longer available. Its functio...,Tools,0
1,COLOURlovers API,From their site: With the release of the COLO...,Tools,0
2,GitHub API,GitHub is a repository of public open source ...,Tools,0
3,uClassify API,The uClassify API allows you to classify text...,Tools,0
4,Pusher API,"Pusher is a simple hosted API for quickly, ea...",Tools,0


In [3]:
df['encoded_text']= df['Category'].astype('category').cat.codes
df.head()

,API,Description,Category,Categoryl,encoded_text
0,Google App Engine API,[This API is no longer available. Its functio...,Tools,0,15
1,COLOURlovers API,From their site: With the release of the COLO...,Tools,0,15
2,GitHub API,GitHub is a repository of public open source ...,Tools,0,15
3,uClassify API,The uClassify API allows you to classify text...,Tools,0,15
4,Pusher API,"Pusher is a simple hosted API for quickly, ea...",Tools,0,15


In [4]:
data = df[['Description','encoded_text']]
data.head()

,Description,encoded_text
0,[This API is no longer available. Its functio...,15
1,From their site: With the release of the COLO...,15
2,GitHub is a repository of public open source ...,15
3,The uClassify API allows you to classify text...,15
4,"Pusher is a simple hosted API for quickly, ea...",15


In [5]:
data.encoded_text.value_counts()

4     884
15    815
7     614
8     588
19    577
13    482
3     453
6     425
5     401
10    380
12    335
14    332
2     314
9     300
1     284
18    282
11    281
16    277
0     254
17    253
Name: encoded_text, dtype: int64

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
import re
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize

data = data.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string

        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing.
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
data['Description'] = data['Description'].apply(clean_text)

In [8]:
texts = data['Description'].tolist()
labels = data['encoded_text'].tolist()

In [9]:
from sklearn.model_selection import train_test_split
texts_trainval, texts_test, labels_trainval, labels_test = train_test_split(texts, labels, test_size=0.2, random_state=42)
texts_train, texts_val, labels_train, labels_val = train_test_split(texts_trainval, labels_trainval, test_size=0.2, random_state=42)


In [10]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture

In [11]:
def tokenize_texts(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='tf')
    input_ids = inputs['input_ids']
    attention_mask = tf.ones_like(input_ids)
    return input_ids, attention_mask

In [12]:
input_ids_train, attention_mask_train = tokenize_texts(texts_train)
input_ids_val, attention_mask_val = tokenize_texts(texts_val)
input_ids_test, attention_mask_test = tokenize_texts(texts_test)


In [13]:
batch_size = 4 # or smaller if needed

# Limit sequence length
max_seq_length = 32  # or smaller if needed
input_ids_train = input_ids_train[:, :max_seq_length]
attention_mask_train = attention_mask_train[:, :max_seq_length]
input_ids_val = input_ids_val[:, :max_seq_length]
attention_mask_val = attention_mask_val[:, :max_seq_length]
input_ids_test = input_ids_test[:, :max_seq_length]
attention_mask_test = attention_mask_test[:, :max_seq_length]

In [14]:
outputs_train = bert_model(input_ids_train, attention_mask=attention_mask_train)


In [15]:
outputs_val = bert_model(input_ids_val, attention_mask=attention_mask_val)
outputs_test = bert_model(input_ids_test, attention_mask=attention_mask_test)

In [16]:
last_hidden_state_train = outputs_train.last_hidden_state
last_hidden_state_val = outputs_val.last_hidden_state
last_hidden_state_test = outputs_test.last_hidden_state

In [17]:
np.save('train_embeddings.npy', last_hidden_state_train.numpy())
np.save('val_embeddings.npy', last_hidden_state_val.numpy())
np.save('test_embeddings.npy', last_hidden_state_test.numpy())

In [30]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
def create_model():
    input_layer = Input(shape=(None, last_hidden_state_train.shape[-1]))
    lstm_layer = LSTM(128)(input_layer)
    output_layer = Dense(20, activation='softmax')(lstm_layer)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [31]:
classifier = create_model()

# Compile model
classifier.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
classifier.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None, 768)]       0         
                                                                 
 lstm_3 (LSTM)               (None, 128)               459264    
                                                                 
 dense_3 (Dense)             (None, 20)                2580      
                                                                 
Total params: 461,844
Trainable params: 461,844
Non-trainable params: 0
_________________________________________________________________


In [32]:
print(type(last_hidden_state_train))
print(type(labels_train))

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [33]:
labels_train = tf.convert_to_tensor(labels_train)


In [34]:
print(type(last_hidden_state_train))
print(type(labels_train))

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [35]:
labels_val = tf.convert_to_tensor(labels_val)
labels_test = tf.convert_to_tensor(labels_test)


In [38]:
classifier.fit(last_hidden_state_train, labels_train, validation_data=(last_hidden_state_val, labels_val), epochs=20, batch_size=128)


Epoch 1/20
43/43 [==============================] - 12s 269ms/step - loss: 0.5082 - accuracy: 0.8485 - val_loss: 1.5018 - val_accuracy: 0.5714
Epoch 2/20
43/43 [==============================] - 11s 268ms/step - loss: 0.4267 - accuracy: 0.8787 - val_loss: 1.5475 - val_accuracy: 0.5795
Epoch 3/20
43/43 [==============================] - 12s 284ms/step - loss: 0.3619 - accuracy: 0.8952 - val_loss: 1.6382 - val_accuracy: 0.5458
Epoch 4/20
43/43 [==============================] - 11s 258ms/step - loss: 0.3284 - accuracy: 0.9091 - val_loss: 1.6580 - val_accuracy: 0.5626
Epoch 5/20
43/43 [==============================] - 10s 238ms/step - loss: 0.2693 - accuracy: 0.9297 - val_loss: 1.6441 - val_accuracy: 0.5766
Epoch 6/20
43/43 [==============================] - 12s 271ms/step - loss: 0.2457 - accuracy: 0.9328 - val_loss: 1.7161 - val_accuracy: 0.5626
Epoch 7/20
43/43 [==============================] - 11s 268ms/step - loss: 0.2068 - accuracy: 0.9496 - val_loss: 1.7725 - val_accuracy: 0.5700

In [39]:
loss, accuracy = classifier.evaluate(last_hidden_state_test, labels_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

54/54 [==============================] - 2s 36ms/step - loss: 2.3304 - accuracy: 0.5536
Test Loss: 2.3303592205047607, Test Accuracy: 0.5536028146743774
